In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
from nltk import tokenize
import boto3
import botocore
from datetime import datetime
import matplotlib.pylab as plt
import yfinance as yf
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from numba import jit, prange

In [3]:
# Load appropriate packages for sentiment analysis
from flair.models import TextClassifier
from flair.models import SequenceTagger
from flair.data import Sentence
from segtok.segmenter import split_single
import pandas as pd
import re


classifier = TextClassifier.load("en-sentiment")

2021-03-02 20:11:55,065 loading file C:\Users\Chris\.flair\models\sentiment-en-mix-distillbert_3.1.pt


In [5]:
nvidia_cleaned_df = pd.read_csv("../00_data/sentiment_test.gz", compression="gzip")

Vader Sentiment

In [6]:
# load vader sentiment for comparing the uncleaned strings to the modified ones
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [29]:
nvidia_cleaned_df.columns

Index(['Unnamed: 0', 'Index', 'videoID', 'datePub', 'searchedDate',
       'VideoTitle', 'channelTitle', 'viewCount', 'likeCount', 'dislikeCount',
       'captionString'],
      dtype='object')

In [33]:
print(f'Title to score = {nvidia_cleaned_df["captionString"][5]}')
score = sid.polarity_scores(str(nvidia_cleaned_df["captionString"][5]))
print(score)

Title to score = (clapboard sounds) - Did you guys just seethe same thing I did? RTX 3070 is gonna outperform a 2080Ti at less than half the price, 499. Is Nvidia insane. I mean, even with all of the leaks indicating that Ampere wasgonna be super powerful, I was in no way ready for this, so okay. (sighs) Deep breath, there's alot to get through here, so let's get started. After an F in the chat forany recent 2080Ti buyers. (giggles) Ouch, like dang. Oh and also our sponsor. Ridge Wallet wants to redefine the wallet with its compact frameand RFID blocking plates. Check out how they can helpkeep your wallet bulge down and use our offer code Linus to save 10% and getfree worldwide shipping. (upbeat music) Now, obviously I'm notready to hand Nvidia their crown until we seethe new cards for ourselves. But based on what they've shown, we are in for one heck of a ride. They announced well a lot andwe're gonna get to all of it, but the big one is, Ampere is real and Ampere is an absolute monst

In [52]:
# utilize regular expressions to split large stings into sentences that are centered around the key words
import re

text = " allet wants to redefine the wallet with its compact frameand RFID blocking plates. Check out how they can helpkeep your wallet bulge down and use our offer code Linus to save 10% and getfree worldwide shipping. (upbeat music) Now, obviously I'm notready to hand Nvidia their crown until we seethe new cards for ourselves. But based on what they've shown, we are in for one heck of a ride. They announced well a lot andwe're gonna get to all of it, but the big one is, Ampere is real and Ampere is an absolute monster. As it turns out, the recentleaks were accurate in many if not most ways."


def search(target, text, context=6):
    # It's easier to use re.findall to split the string,
    # as we get rid of the punctuation
    words = re.findall(r"\w+", text)

    matches = (i for (i, w) in enumerate(words) if w.lower() == target)
    for index in matches:
        if index < context // 2:
            yield words[0 : context + 1]
        elif index > len(words) - context // 2 - 1:
            yield words[-(context + 1) :]
        else:
            yield words[index - context // 2 : index + context // 2 + 1]


print(list(search("nvidia", text, 10)))

[['I', 'm', 'notready', 'to', 'hand', 'Nvidia', 'their', 'crown', 'until', 'we', 'seethe']]


In [53]:
search("nvidia", text, 15)

<generator object search at 0x00000201143A05C8>

In [59]:
test_list_to_str = list(search("nvidia", text, 30))
test_list_to_str

[['save',
  '10',
  'and',
  'getfree',
  'worldwide',
  'shipping',
  'upbeat',
  'music',
  'Now',
  'obviously',
  'I',
  'm',
  'notready',
  'to',
  'hand',
  'Nvidia',
  'their',
  'crown',
  'until',
  'we',
  'seethe',
  'new',
  'cards',
  'for',
  'ourselves',
  'But',
  'based',
  'on',
  'what',
  'they',
  've']]

In [60]:
new_str = " ".join(test_list_to_str[0])

In [61]:
score = sid.polarity_scores(new_str)
print(score)

{'neg': 0.0, 'neu': 0.808, 'pos': 0.192, 'compound': 0.7506}


### Flair Sentiment

In [64]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load("en-sentiment")
sentence = Sentence(new_str)
classifier.predict(sentence)
# print sentence with predicted labels
print("Sentence above is: ", sentence.labels)

2021-03-02 20:27:50,602 loading file C:\Users\Chris\.flair\models\sentiment-en-mix-distillbert_3.1.pt
Sentence above is:  [POSITIVE (0.8555)]
